[View in Colaboratory](https://colab.research.google.com/github/Hoiy/kaggle-avito-demand-prediction/blob/master/prep_fasttext.ipynb)

In [0]:
import pandas as pd
import numpy as np
import os
import dotenv
import fastText
from tqdm import tqdm 

dotenv.load_dotenv('.env')
tqdm.pandas()
types = ['train', 'test']

In [0]:
for key in types:
  path = './data/%s_prep.snappy.parquet'%key
  !gsutil cp gs://{os.environ['GCP_BUCKET']}/{key}_prep.snappy.parquet {path}

In [0]:
ft = fastText.load_model('./data/russian_fasttext.bin')

In [0]:
dfs = {t:pd.read_parquet('./data/%s_prep.snappy.parquet'%t) for t in types}
[dfs[t].shape for t in types]

In [0]:
# Fasttext Encoding

for col in ['title']:
  for t in types:
    index = [col+'_ft_%d'%i for i in range(ft.get_dimension())]
    dfs[t] = pd.concat([dfs[t][col], dfs[t][col].progress_apply(lambda x: pd.Series(ft.get_sentence_vector(x), index=index))], axis=1)

In [0]:
for key in dfs:
  path = './data/%s_prep_2.snappy.parquet'%key
  dfs[key].to_parquet(path)
  !gsutil cp {path} gs://{os.environ['GCP_BUCKET']}/{key}_prep_2.snappy.parquet